# Image color quantization BNHP KMeans
We'll compress an image by applying our implementation of Kmeans to its pixels to find a small number of RGB combinations which, if we replace each pixel by the nearest of these combination, we'll still have a good image quality.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist as distance
from mpl_toolkits.mplot3d import Axes3D 
np.random.seed(10)

In [ ]:
def BNHP_KmeansWithDistortion(X, K, means=[], NumIterations = 20):
    # Init: for simplicity, we'll take the first K data points to initialize the cluster means
    # You can try replacing it with other initialization, and see what happens
    if len(means)==0:
        means = X[:K]
    # The code below implements the core logic of the Kmeans algorithm
    for _ in range(NumIterations):
        # line 1: compute the distance between each sample and every cluster center
        distances = distance(X,means)
        # line 2: set the samples of group k to be all the points closest to Ck
        labels = distances.argmin(axis=1)
        # line 3: for each k set Ck to be the mean of all points in group k 
        means = [(X[labels==i].mean(axis=0)) for i in range(len(means))]
        
    distortion = [distance(X[labels==k],means[k].reshape(1,X.shape[1])).sum() for k in range(K)]
    return means, labels, distortion

In [ ]:
# load the image
img = plt.imread("elephant.jpg")

# divide by 255 so that the data is between 0 and 1, for plt.imshow
img = np.array(img) / 255

# plot it
plt.imshow(img);

In [ ]:
# reshape the image into a 2D numpy array
width, height, channels = img.shape
X = np.reshape(img, (width * height, channels))# note that pixel in the image is now a row in X

In [ ]:
# plot some of the pixels in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ind = np.random.permutation(X.shape[0])[:1000]# NOTE - we randomly choose 1000 pixels to draw
ax.scatter(X[ind,0],X[ind,1],X[ind,2], c=X[ind])

ax.set_xlabel('R')
ax.set_ylabel('G')
ax.set_zlabel('B');

In [ ]:
# lets cluster the 3D pixels into K=4 groups 
K = 4

# we randomly choos K pixels as our initial guess of the centroids
uX = np.unique(X,axis=0)
ndx = np.random.permutation(uX.shape[0])
RGBinit = uX[ndx[:K]]

# apply Kmeans to the data
means, labels, distortions = BNHP_KmeansWithDistortion(X, K, means = RGBinit,NumIterations=100)

To plot the image after replacing each pixel by the centroid of it's cluster, we need to reshape the 2D data `X` into the original 3D image with shape `width x height x channels`

In [ ]:
# restore image from pixels and image dimensions
def Pixels2Image(means, labels, width, height):
    image = np.zeros((width, height, channels))
    label_idx = 0
    for i in range(width):
        for j in range(height):
            image[i][j] = means[labels[label_idx]]
            label_idx += 1
    return image

# plot original image
plt.figure()
plt.axis('off')
plt.title('Original')
plt.imshow(img)

# and also the compressed image
plt.figure()
plt.axis('off')
plt.title('K='+str(K)+' colors')
plt.imshow(Pixels2Image(means, labels, width, height));

## Learning to cluster using a sample of the data

Images can have many pixels, so we might want to learn clusters from a subset of pixels, and then cluster the entire data based on these centroids.
**Exercise:** complete the missing line below

In [ ]:
# train on sample of pixels, then label all pixels according to their min distance to centroids, then restore the shape and display compressed image
#

# first, randomly choose some pixels to train on 
ind = np.random.permutation(X.shape[0])
Xtrain = X[ind[:1000]]

# init
#uX = np.unique(Xtrain,axis=0)
#ndx = np.random.permutation(uX.shape[0])
K = np.array([2,3,4,8,32])
RGBinit = uX[ndx[:K.max()]]
 
# plot original image
plt.figure()
plt.axis('off')
plt.title('Original')
plt.imshow(img)

for k in K:
    # kmeans on sample of pixels
    means, labels, distortions = BNHP_KmeansWithDistortion(Xtrain, k, means = RGBinit[:k],NumIterations=200)
    # associating every pixel with one of the clusters

    ###### COMPLETE HERE THE LABELING OF ALL POINTS, INCLUDING THOSE WHICH WERE NOT PART OF THE CLUSTERED DATA#####
    labels = 
    ###############################################################################################################

    # dsplaying the compressed image
    plt.figure()
    plt.axis('off')
    plt.title('K='+str(k)+' colors')
    plt.imshow(Pixels2Image(means, labels, width, height))

**Exercise:** 

1. Repeat the compression procedure above with each of the following images 
    - rgb.jpg
    - BNHP.jpg
    - Apples.jpg
2. What can be said, for each image, regarding 
    - the 3D representation of its pixels
    - the number of colors required for a compressed image which is reasonably similar to the original image
2. Download an image from the internet and repeat the process

In [ ]:
# load the image
img = plt.imread("Apples.jpg")

# divide by 255 so that the data is between 0 and 1, for plt.imshow
img = np.array(img) / 255

# reshape the image into a 2D numpy array
width, height, channels = img.shape
X = np.reshape(img, (width * height, channels))# note that pixel in the image is now a row in X


# train on sample of pixels, then label all pixels according to their min distance to centroids, then restore the shape and display compressed image
#============

# first, randomly choose some pixels to train on 
np.random.seed(10)
ind = np.random.permutation(X.shape[0])
Xtrain = X[ind[:1000]]

# init
uX = np.unique(Xtrain,axis=0)
ndx = np.random.permutation(uX.shape[0])
K = np.array([2,3,4,8,32])
RGBinit = uX[ndx[:K.max()]]
 
# plot original image
plt.figure()
plt.axis('off')
plt.title('Original')
plt.imshow(img)

# plot some of the pixels in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ind = np.random.permutation(X.shape[0])[:1000]# NOTE - we randomly choose 1000 pixels to draw
ax.scatter(X[ind,0],X[ind,1],X[ind,2], c=X[ind])

ax.set_xlabel('R')
ax.set_ylabel('G')
ax.set_zlabel('B');


for k in K:
    # kmeans on sample of pixels
    means, labels, distortions = BNHP_KmeansWithDistortion(Xtrain, k, means = RGBinit[:k],NumIterations=200)
    # associating every pixel with one of the clusters
    labels = ### COPY THE LINE YOU HAVE COMPLETED ABOVE

    # dsplaying the compressed image
    plt.figure()
    plt.axis('off')
    plt.title('K='+str(k)+' colors')
    plt.imshow(Pixels2Image(means, labels, width, height))